In [28]:
import tensorflow as tf
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

tf.compat.v1.disable_eager_execution()
# 모델을 훈련하지 않습니다., 훈련 연산을 비활성화
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights = 'imagenet',
                                include_top = False)

# 손실 계산 여러 층에 있는 모든 필터 활성화를 동시 시행
# 어떤 층들을 선택했는지에 따라 만들어내는 시각요소에 큰 영향을 끼칩니다,

# 층 이름과 계수를 매핑한 딕셔너리, 최대화 하려는 손실에 층의 활성화가 기여할 양을 정합니다.
layer_contributions = {
    'mixed2':0.2,
    'mixed3':3.,
    'mixed4':2.,
    'mixed5':5
}

C:\Users\c\.conda\envs\pytorch\lib\site-packages\keras\backend.py:414: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [29]:
# 손실 텐서 정의
# L2 노름의 가중치 합
layer_dict = dict([(layer.name, layer) for layer in model.layers])

# loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    # 층의 출력층 얻기
    activation = layer_dict[layer_name].output 
    
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    # 층 특성의 L2 노름 제곱을 손실에 추가합니다. 
    loss =   K.variable(0.) + (coeff * K.sum(K.square(activation[:, 2: -2, 2:-2, :])) / scaling)

In [30]:
dream = model.input  # 이미지 저장

grads = K.gradients(loss, dream)[0]  # 손실에 대한 딥드림 이미지의 그레디언트 계산

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)    # 그레디언트 정규화
# 손실과 gradient 값을 계산할 케라스 함수 생성
outputs = [loss , grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

# 경사 상승법 여러번 반복
def gradient_ascent(x, iterations, step, max_loss = None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print(i, '번째 손실 : ', loss_value)
        x += step * grad_values
    return x

In [31]:
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
              float(size[0]) / img.shape[1],
              float(size[1]) / img.shape[2],
                   1)
    return scipy.ndimage.zoom(img, factors, order = 1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis = 0)
    img = inception_v3.preprocess_input(img)
    return img
    
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [32]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10

base_image_path = './original_photo_deep_dream.jpg'
img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
succesive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** 1))
                  for dim in original_shape])
    succesive_shapes.append(shape)

succesive_shapes = succesive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, succesive_shapes[0])

for shape in succesive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                         iterations = iterations,
                         step = step,
                         max_loss = max_loss)
    
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img , fname = 'dream_at_scale_' + str(shape) + '.png')

save_img(img, fname = './final_dream.png')

처리할 이미지 크기 (250, 250)
0 번째 손실 :  0.38168335
1 번째 손실 :  0.58073276
2 번째 손실 :  0.80944896
3 번째 손실 :  1.0608652
4 번째 손실 :  1.3113792
5 번째 손실 :  1.6232666
6 번째 손실 :  2.023442
7 번째 손실 :  2.513495
8 번째 손실 :  2.9592946
9 번째 손실 :  3.6489203
10 번째 손실 :  4.356871
11 번째 손실 :  4.6516924
12 번째 손실 :  5.6578035
13 번째 손실 :  6.341509
14 번째 손실 :  7.1342974
15 번째 손실 :  7.8280973
16 번째 손실 :  8.785484
17 번째 손실 :  9.337323
처리할 이미지 크기 (250, 250)
처리할 이미지 크기 (350, 350)
0 번째 손실 :  0.7656622
1 번째 손실 :  1.8136775
2 번째 손실 :  2.9394631
3 번째 손실 :  3.965658
4 번째 손실 :  5.1302085
5 번째 손실 :  6.3761024
6 번째 손실 :  7.8313046
7 번째 손실 :  8.751801
